In [ ]:
def load_properties(file_path):
    """
    Load properties from a given property file.

    Args:
        file_path (str): Path to the property file.

    Returns:
        dict: A dictionary with property key-value pairs.
    """
    properties = {}
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith("#"):  # Skip comments and empty lines
                key, value = line.split("=", 1)
                properties[key.strip()] = value.strip()
    return properties



import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import os
import cv2
import numpy as np
import mysql.connector

config = load_properties("config.properties")


# Callback to resize background image
def resize_background(event=None):
    global bg_photo  # Declare bg_photo as global to persist the reference

    # Get the current dimensions of the canvas
    canvas_width = window.winfo_width()
    canvas_height = window.winfo_height()

    # Resize the background image to match the canvas dimensions
    resized_image = bg_image.resize((canvas_width, canvas_height), Image.Resampling.LANCZOS)
    bg_photo = ImageTk.PhotoImage(resized_image)

    # Update the canvas with the resized image
    canvas.delete("all")  # Clear all items from the canvas
    canvas.create_image(0, 0, image=bg_photo, anchor="nw")
    canvas.lower("all")  # Ensure the background stays at the lowest layer

# Function to show the register form and refresh the canvas
# Function to show the register form with updated styling
def show_register_form():
    global form_frame, t1, t2, t3, submit_button

    # Create the form frame with modern style
    form_frame = tk.Frame(
        window,
        bg="#FFFFFF",  # White background for a clean look
        bd=5,
        relief="flat",  # Flat border for a modern feel
        highlightbackground="#E8E8E8",  # Lighter border color
        highlightthickness=1,
        padx=30,  # Add padding around the content
        pady=20,  # Add padding around the content
    )
    form_frame.place(relx=0.25, rely=0.25, anchor="center", width=600, height=300)

    label_font = ("Helvetica", 14,"bold")
    entry_font = ("Helvetica", 12)

    # Create form labels with modern styling
    tk.Label(form_frame, text="Name:", font=label_font, bg="white", fg="#333333").grid(
        row=0, column=0, padx=20, pady=10, sticky="w"
    )
    t1 = tk.Entry(form_frame, font=entry_font, width=40, bg="#f7f7f7", bd=2, relief="solid", highlightthickness=1)
    t1.grid(row=0, column=1, padx=20, pady=10)

    tk.Label(form_frame, text="Age:", font=label_font, bg="white", fg="#333333").grid(
        row=1, column=0, padx=20, pady=10, sticky="w"
    )
    t2 = tk.Entry(form_frame, font=entry_font, width=40, bg="#f7f7f7", bd=2, relief="solid", highlightthickness=1)
    t2.grid(row=1, column=1, padx=20, pady=10)

    tk.Label(form_frame, text="Address:", font=label_font, bg="white", fg="#333333").grid(
        row=2, column=0, padx=20, pady=10, sticky="w"
    )
    t3 = tk.Entry(form_frame, font=entry_font, width=40, bg="#f7f7f7", bd=2, relief="solid", highlightthickness=1)
    t3.grid(row=2, column=1, padx=20, pady=10)

    # Create the submit button with modern styling
    submit_button = tk.Button(
        form_frame,
        text="Submit",
        font=("Helvetica", 14, "bold"),
        bg="#4CAF50",  # Fresh green color for action buttons
        fg="white",
        width=20,
        bd=0,
        relief="flat",
        activebackground="#45a049",  # Darker green when active
        command=submit_form,
        padx=10,
        pady=5
    )
    submit_button.grid(row=3, column=0, columnspan=2, pady=20)

    # Add soft shadow effect to the form frame (optional)
    form_frame.config(highlightthickness=2, highlightbackground="#CCCCCC", relief="solid")

    # Force canvas and background refresh
    window.update_idletasks()
    resize_background()

# Initialize main window
window = tk.Tk()
window.title("Face Recognition System")
window.geometry("1920x1080")  # Set the default size to match the background image

# Load and configure the background image
try:
    image_path = os.path.join(os.getcwd(), "img6.jpg")
    bg_image = Image.open(image_path)
except FileNotFoundError:
    print(config["error.background"]) 
    bg_image = None  # Prevent crashes if the image is missing
except Exception as e:
    print(f"Error: {e}")
    bg_image = None

# Create canvas for the background
canvas = tk.Canvas(window)
canvas.pack(fill="both", expand=True)

# Bind resize event for dynamic resizing
if bg_image:
    window.bind("<Configure>", resize_background)

# Header Section
header = tk.Frame(window, bg="#004d80", height=80)
header.pack(fill="x")
header_label = tk.Label(
    header, text="FACE RECOGNITION SYSTEM", font=("Helvetica", 28, "bold"), fg="white", bg="#004d80"
)
header_label.pack(pady=20)


# Function to handle form submission
def submit_form():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showinfo("Result", config["error.missing_details"])
        return

    # Run the generate_and_train function
    generate_and_train()

    # Destroy the form and button after successful submission
    form_frame.destroy()
    messagebox.showinfo("Result", config["info.registration_success"])


# Function to generate and train data
def generate_and_train():
    # Establish MySQL connection
    mydb = mysql.connector.connect(
        host=config["db.host"],
        user=config["db.user"],
        password=config["db.password"],
        database=config["db.database"],
    )
    mycursor = mydb.cursor()

    # Check if the name already exists in the database
    name_check_query = f"SELECT * FROM {config['db.table']} WHERE Name = %s"
    mycursor.execute(name_check_query, (t1.get(),))
    existing_user = mycursor.fetchone()

    if existing_user:
        messagebox.showerror("Error", "Name already exists in the database!")
        return

    # Insert new user
    mycursor.execute(f"SELECT * from {config['db.table']}")
    myresult = mycursor.fetchall()
    id = len(myresult) + 1
    sql = f"INSERT INTO {config['db.table']} (id, Name, Age, Address) VALUES (%s, %s, %s, %s)"
    val = (id, t1.get(), t2.get(), t3.get())
    mycursor.execute(sql, val)
    mydb.commit()

    # Step 1: Generate Dataset
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y : y + h, x : x + w]
        return cropped_face

    cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = os.path.join(config["data_directory"], f"user.{id}.{img_id}.jpg")
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face)

        if cv2.waitKey(1) == 13 or img_id == 200:
            break

    cap.release()
    cv2.destroyAllWindows()

    # Step 2: Train Classifier
    data_dir = config["data_directory"]  # Use the path from config
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert("L")
        image_np = np.array(img, "uint8")
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(image_np)
        ids.append(id)

    ids = np.array(ids)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")


# Buttons for functionalities
button_style = {"font": ("Helvetica", 14, "bold"), "bg": "#004d80", "fg": "white", "width": 20, "bd": 3}

b_combined = tk.Button(window, text="Register New User", **button_style, command=show_register_form)
b_combined.place(relx=0.25, rely=0.6, anchor="center")

def detect_face():

    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        # Convert to grayscale and apply histogram equalization
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_img = cv2.equalizeHist(gray_img)
        
        # Detect faces
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
        
        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            
            # Predict the identity of the person
            id, pred = clf.predict(gray_img[y:y + h, x:x + w])
            confidence = int(100 * (1 - pred / 300))

            mydb=mysql.connector.connect(
            host=config["db.host"],
            user=config["db.user"],
            password=config["db.password"],
            database=config["db.database"],
            )
            mycursor=mydb.cursor()
           # Inside your detect_face function

           # Fetch name from the database
            mycursor.execute(f"SELECT name FROM {config['db.table']} WHERE id = %s", (id,))

            s = mycursor.fetchone()

            # Check if s is None, and if so, set it to "UNKNOWN"
            if s is None:
                s = "UNKNOWN"
            else:
                s = ''.join(s) if isinstance(s, tuple) else str(s)

            # Display name with confidence
            if confidence > 75:  # Higher threshold for more accurate recognition
                cv2.putText(img, f"{s} ({confidence}%)", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        return img
    
    # Load face detector and LBPH classifier with modified parameters
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create(radius=2, neighbors=8, grid_x=8, grid_y=8)
    clf.read("classifier.xml")
    
    # Capture video from webcam
    video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    
    while True:
        ret, img = video_capture.read()
        img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("Face Detection", img)
    
        if cv2.waitKey(1) == 13:  # Press Enter key to exit
            break
    
    video_capture.release()
    cv2.destroyAllWindows()



b_detect = tk.Button(window, text="Detect Faces", **button_style, command=detect_face)
b_detect.place(relx=0.25, rely=0.5, anchor="center")

def train_classifier():
    data_dir = config["data_directory"]  # Use the path from config
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result", config["info.training_complete"])



b_refresh = tk.Button(window, text="Refresh", **button_style, command=train_classifier)
b_refresh.place(relx=0.25, rely=0.7, anchor="center")

# Footer Section
footer = tk.Frame(window, bg="#004d80", height=10)
footer.pack(fill="x", side="bottom")

# Main event loop
window.geometry("1000x700")
window.mainloop()
